# Sample notebook for Azure OpenAI 


First connect to the Azure OpenAI service. You will need to create a resource in Azure and get the key and endpoint. 


In [ ]:
!pip install openai
!pip install python-dotenv
!pip install azure-cosmos

In [ ]:
import os
import openai
from dotenv import load_dotenv

import json
from azure.cosmos import CosmosClient, PartitionKey
import uuid

In [ ]:
load_dotenv()

openai.api_type = "azure"
openai.api_base = "https://openai79.openai.azure.com/"
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  # engine="gpt-35-turbo",
  engine="text-davinci-003",    
  prompt="---\n# Extract the following from the below text:\n - Extract Item for Sale, Location and Price.\n - Extract the aspect based sentiment and the overall sentiment.\n - Aspects are like Delivery, Communication, Packaging, Time, Price, etc.\n - Sentiment should be positive, negative or neutral for both the aspect based and the overall sentiment.\n---\n# Be concise\n# DO NOT INCLUDE CODE\n# format the output in a valid JSON format\n---\n24\" Computer monitor, $50 local pickup in Boise ID. Used for two years but worked great, I just upgraded my setup. the delivery wasn't great though\n",
  temperature=1,
  max_tokens=500,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0)

print(response.choices[0].text.strip())

Store the output in a JSON file and upload it to Cosmos DB using the Azure Cosmos DB Notebook extension.

In [ ]:
# connect to cosmos db

load_dotenv()

ENDPOINT = os.getenv("COSMOS_ENDPOINT")
KEY = os.getenv("COSMOS_KEY")
DATABASE_NAME = "absadb"
CONTAINER_NAME = "sentiments"

client = CosmosClient(url=ENDPOINT, credential=KEY)
database = client.create_database_if_not_exists(id=DATABASE_NAME)
print("Database\t", database.id)

key_path = PartitionKey(path="/id")
container = database.create_container_if_not_exists(
    id=CONTAINER_NAME, partition_key=key_path, offer_throughput=400
)
print("Container\t", container.id)

In [ ]:
# add item to cosmos db container
z = json.loads(response.choices[0].text.strip())
y = {"id": str(uuid.uuid4())}
z.update(y)

container.create_item(z)